## Задание 1

In [70]:
import pandas as pd
import sklearn as sk
import math
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from razdel import tokenize as razdel_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
import numpy as np

In [6]:
data = pd.read_csv('labeled.csv')
data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [7]:
data.toxic.value_counts(normalize=True)

0.0    0.66514
1.0    0.33486
Name: toxic, dtype: float64

In [9]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)

train.reset_index(inplace=True)
test.reset_index(inplace=True)

### дефолтная токенизация

In [10]:
vectorizer = CountVectorizer() 

In [11]:
x = vectorizer.fit_transform(train.comment)
x_test = vectorizer.transform(test.comment) 

y = train.toxic.values
y_test = test.toxic.values

In [16]:
clf = LogisticRegression(C=0.1, class_weight='balanced')
clf.fit(x, y)
pred = clf.predict(x_test)

In [17]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

         0.0       0.91      0.83      0.87       954
         1.0       0.71      0.84      0.77       488

    accuracy                           0.83      1442
   macro avg       0.81      0.83      0.82      1442
weighted avg       0.84      0.83      0.83      1442



### razdel.tokenize

In [26]:
def razdel_tokenize(text):
    tokens = list(tokenize(text))
    tokens = [token.text for token in tokens]
    return tokens

In [27]:
razdel_vectorizer = CountVectorizer(tokenizer = razdel_tokenize)
x2 = razdel_vectorizer.fit_transform(train.comment)
x2_test = razdel_vectorizer.transform(test.comment)
y2 = train.toxic.values
y2_test = test.toxic.values

In [30]:
clf_razdel = LogisticRegression(C=0.1, class_weight='balanced')
clf_razdel.fit(x2, y2)
pred_razdel = clf_razdel.predict(x2_test)

C:\Users\howto\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [34]:
print(classification_report(y2_test, pred_razdel))

              precision    recall  f1-score   support

         0.0       0.91      0.84      0.87       954
         1.0       0.73      0.83      0.78       488

    accuracy                           0.84      1442
   macro avg       0.82      0.84      0.82      1442
weighted avg       0.85      0.84      0.84      1442



**razdel.tokenize показывает лучший результат, но незначительно**

## Задание 2

In [50]:
text = ['я и ты', 'ты и я', 'я, я и только я', 'только не я', 'он']

In [57]:
def tf(term, sent):
    tf = []
    tf = sent.count(term) / len(sent.split())
    return tf

def idf(term, text):
        df = 0
        idf = []
        for d in text:
            if term in d:
                df += 1
                idf = math.log(len(text) / df) 
        return idf
    
def tf_idf(term, sent, text):
    tfidf = []
    tfidf = tf(term, sent) * idf(term, text)
    return tfidf

In [58]:
vocabulary = ['я', 'ты', 'и', 'только', 'не', 'он']

In [61]:
table_tfidf = pd.DataFrame({"я" : [1, 1, 3, 1, 0],
                            "ты" : [1, 1, 0, 0, 0],
                            "и" : [1, 1, 1, 0, 0],
                            "только" : [0, 0, 1, 1, 0],
                            "не" : [0, 0, 0, 1, 0],
                            "он" : [0, 0, 0, 0, 1]})

In [62]:
for s in voc:
    x = []
    for d in text:
        x.append(tf_idf(s, d, text))
    table_tfidf[s] = x
    
table_tfidf.index = text
print(table_tfidf)

                        я       ты         и    только        не        он
я и ты           0.074381  0.30543  0.170275  0.000000  0.000000  0.000000
ты и я           0.074381  0.30543  0.170275  0.000000  0.000000  0.000000
я, я и только я  0.133886  0.00000  0.102165  0.183258  0.000000  0.000000
только не я      0.074381  0.00000  0.000000  0.305430  0.536479  0.000000
он               0.000000  0.00000  0.000000  0.000000  0.000000  1.609438


## Задание 3

In [103]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 1), lowercase=True, stop_words = None, encoding="utf-8")
X_1 = vectorizer.fit_transform(train.comment)
X_1_test = vectorizer.transform(test.comment) 

y_1 = train.toxic.values
y_1_test = test.toxic.values

clf = DecisionTreeClassifier(max_depth=8, class_weight='balanced')
clf.fit(X_1, y_1)
dtc_preds = clf.predict(X_1_test)

print(classification_report(y_1_test, dtc_preds))

              precision    recall  f1-score   support

         0.0       0.79      0.59      0.68       961
         1.0       0.46      0.69      0.55       481

    accuracy                           0.62      1442
   macro avg       0.63      0.64      0.61      1442
weighted avg       0.68      0.62      0.63      1442



In [105]:
tfidf_vectorizer = TfidfVectorizer(min_df=10, max_df=0.4, lowercase = False, ngram_range = (1,2))
X1 = vectorizer.fit_transform(train.comment)
X1_test = vectorizer.transform(test.comment) 

y1 = train.toxic.values
y1_test = test.toxic.values

clf_MNB = MultinomialNB(alpha = 1, fit_prior = True)
clf_MNB.fit(X1, y1)
mnb_preds = clf.predict(X1_test)

print(classification_report(y1_test, mnb_preds))

              precision    recall  f1-score   support

         0.0       0.79      0.59      0.68       961
         1.0       0.46      0.69      0.55       481

    accuracy                           0.62      1442
   macro avg       0.63      0.64      0.61      1442
weighted avg       0.68      0.62      0.63      1442



In [87]:
def find_toxic(probas):
    probas = [p[1] for p in probas]
    test['probas'] = probas
    
    res = test.sort_values(by = 'probas', ascending = False)[:10]
    res = res.reset_index(drop=True)
    
    for i in range(10):
        print('toxic:', res.loc[i].toxic)
        print('comment: ', res.loc[i].comment)
        return (res)

In [106]:
dtc_preds = clf.predict(X_1_test)
dtc_probas = clf.predict_proba(X_1_test)

In [107]:
print (find_toxic(dtc_probas))

toxic: 1.0
comment:  если вспомнить, что у русских фамилии хохлов, то все сходится

   index                                            comment  toxic  probas
0  14035  если вспомнить, что у русских фамилии хохлов, ...    1.0     1.0
1   2569  АВСИЕВВСТЬИЫФЦВФФСОАЬ! АЛЕКС! ПЫНЯ! СИСЯН! ХОХ...    1.0     1.0
2   2343  Какая убогая банда, гуляет на просторах YouTub...    1.0     1.0
3  13795  Обезьяна, имей совесть! 1nb4: уже поимел! В s ...    1.0     1.0
4   6101  Да какой там у хохлов языг? Можно любую хуету ...    1.0     1.0
5   5132  ЦИФРЫ, блять, не сходятся. Вот как вам отвечат...    1.0     1.0
6  14205  КАКИЕ ХОХЛЫ?! два урода. при чём ту вообще ХОХ...    1.0     1.0
7   1978              граждане России Хохлы какие-нибудь.\n    1.0     1.0
8    247  моча сюда не заходит И как привлекать их внима...    1.0     1.0
9  14078  просто остаётся пройти 3 км Ну-ну давай. Хочеш...    0.0     1.0


<ipython-input-87-69ae0579ced9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['probas'] = probas


In [102]:
print (find_toxic(mnb_probas))

toxic: 1.0
comment:  Рефлексия? Пиво? Хохлы? У тебя галлюцинации?

   index                                            comment  toxic  probas
0  10292     Рефлексия? Пиво? Хохлы? У тебя галлюцинации?\n    1.0     1.0
1   5600  Вот и хохлы подгоревшие подъехали. Куда Ньюсач...    1.0     1.0
2  14253  Надо просто Тарасов-пидорашек из по выгнать, т...    1.0     1.0
3   1965  90 не будет этим заниматься. Хохлов отрезают о...    1.0     1.0
4    931  Если говорит, значит пиздит Это про хохлов ска...    1.0     1.0
5    350  Вы вот смеетесь, а что будет, когда у нас эти ...    1.0     1.0
6   5933  Навязывание любой религии дно, но есть. Правос...    1.0     1.0
7   5803       бедная пидораха, и здесь хохлы ей выноваты\n    1.0     1.0
8   5535  выискивают хохлов, живущих в рашке и по умолча...    1.0     1.0
9  14167                   Не стоит корить хохлов за это.\n    1.0     1.0


<ipython-input-87-69ae0579ced9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['probas'] = probas


В результате, оба классификатора отобрали разные токсичные комментарии; MultinomialNB выбрал более токсичные комментарии, чем DecisionTreeClassifier.